In [1]:
import numpy as np 
import pandas as pd
import scanpy as sc
import anndata
import igraph
import os
sc.settings.verbosity = 3         # verbosity: errors (0), warnings (1), info (2), hints (3)

In [2]:
nucl_genes_50 = ['MALAT1', 'NEAT1', 'FTX', 'FOXP1', 'RBMS3', 'ZBTB20', 'LRMDA', 'PBX1', 'ITPR2', 'AUTS2', 'TTC28', 'BNC2', 'EXOC4', 'RORA', 'PRKG1', 'ARID1B', 'PARD3B', 'GPHN', 'N4BP2L2', 'PKHD1L1', 'EXOC6B', 'FBXL7', 'MED13L', 'TBC1D5', 'IMMP2L', 'SYNE1', 'RERE', 'MBD5', 'EXT1', 'WWOX', 'EPB41L4A', 'PTK2', 'ST6GAL1', 'CHD9', 'PTPRG', 'JMJD1C', 'WSB1', 'SBF2', 'STAG1', 'GMDS', 'ADAMTS9-AS2', 'PDE7B', 'RALGAPA2', 'PRKN', 'ZFAND3', 'PLXDC2', 'NAALADL2', 'ASAP1', 'NFIA', 'MIR99AHG']

### DEFINE GENE SETS
celltype_gene_set_dict = {
"MT" : ['MT-ND1', 'MT-ND2', 'MT-CO1', 'MT-CO2', 'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4', 'MT-ND5', 'MT-ND6', 'MT-CYB'],
"MT_nucl" : ["POLG", "POLG2", "TWNK", "SSBP1", "PRIMPOL", "DNA2", "MGME1", "RNASEH1", "POLRMT", "TFAM", "TEFM", "TFB2M", "TK2", "DGUOK", "RRM2B", "TYMP", "SLC25A4", "OPA1", "MFN1", "MFN2", "DNM1L", "MFF", "FIS1", "MPV17", "SPG7"], 
"CM_cyto" : ["TTN", "RYR2", "PAM", "TNNT2", "RABGAP1L", "PDLIM5", "MYL7", "MYH6"],
"CM_nucl" : ["RBM20", "TECRL", "MLIP", "CHRM2", "TRDN", "PALLD", "SGCD", "CMYA5", "MYOM2", "TBX5", "ESRRG", "LINC02248", "KCNJ3", "TACC2", "CORIN", "DPY19L2", "WNK2", "MITF", "OBSCN", "FHOD3", "MYLK3", "DAPK2", "NEXN"],
"VEC" : ["VWF", "ERG", "ANO2", "PTPRB", "EGFL7", "PREX2", "ADGRL4", "FLT1", "CYYR1", "GRB10", "PPP1R16B", "DOCK9", "SHANK3", "PECAM1", "PLEKHG1", "EMCN"],
"PER" : ["RGS5", "DACH1", "GUCY1A1", "ABCC9", "BGN", "NOTCH3", "PDGFRB", "FRMD3", "RNF152", "CCDC102B", "NGF"],
"SMC" : ["MYH11", "KCNAB1", "NTRK3", "CHRM3", "ACTA2", "RGS6", "DGKG", "ITGA8", "TBX2", "LMOD1", "SDK1", "GPC6", "ANTXR1", "FLNA", "CLMN", "ATP10A", "MCAM", "TAGLN", "CCDC3"],
"AD" : ["PLIN4", "PLIN1", "PDE3B", "GPAM", "PTPRS", "PPARG", "MLXIPL", "MGST1", "AQP7", "SLC19A3", "FABP4", "TPRG1", "DIRC3", "LPL", "PNPLA2", "LIPE", "ADH1B", "ADIPOQ", "PRKAR2B", "CIDEA", "LINC00278", "PFKFB3", "LINC02237", "LIPE-AS1", "SVEP1"],
"SC" : ["XKR4", "AC016766.1", "SLC35F1", "ZNF536", "NCAM2", "GPM6B", "KIRREL3", "SORCS1", "ST6GALNAC5", "PRKCA", "GINS3", "PMP22", "ALDH1A1", "IL1RAPL2", "DOCK5", "NKAIN3", "COL28A1", "RALGPS2", "PKN2-AS1", "KLHL29", "PTPRZ1"],
"N" : ["CSMD1", "SYT1", "KCNIP4", "CNTNAP2", "DLGAP1", "PTPRD", "LRRTM4", "ATRNL1", "LRP1B", "CTNND2", "KCNQ5", "NRG3", "SNTG1", "GRIA2", "RIMS2", "CSMD3", "XIST", "KAZN", "DPP10", "HS6ST3", "OPCML"],
"EEC" : ["PCDH7", "PCDH15", "LINC02147", "LINC02388", "MYRIP", "GMDS", "ADAMTSL1", "LEPR", "CALCRL", "CGNL1", "HMCN1", "NPR3", "POSTN"],
"FB" : ["DCN", "ABCA8", "ABCA6", "ABCA10", "FBLN1", "COL15A1", "FBN1", "C7"],
"L" : ["SKAP1", "RIPOR2", "CD247", "IKZF1", "BCL11B", "SLFN12L", "ITGAL", "SAMD3", "CARD11", "CDC42SE2", "CCND3"],
"MESO" : ["C3", "SULF1", "AP000561.1", "PRG4", "GPM6A", "CDON", "DPP6", "CCDC80", "EZR", "FOS", "BNC1", "AC245041.2", "PRKD1", "CYSTM1", "TLL1", "WT1"],
"MP" : ["TBXAS1", "SLC9A9", "MRC1", "MS4A6A", "RBM47", "DOCK2", "MCTP1", "SYK", "MSR1", "ATP8B4", "F13A1", "CD74", "MS4A4E", "ADAP2"],
"dev_enteroendocrine" : ["RIMS2", "CHGA", "TOX", "PTRN2", "RIMBP2", "RAP1GAP2", "DACH1"],
"dev_2" : ["FLNB", "ANXA1", "LINC00511", "THSD4", "NPEPPS", "KCNMA1", "TGFB1"],
"dev_enterocytes" : ["PATJ", "TSPAN8", "EPCAM", "ABCC3", "LGALS3", "CCSER1", "CDH17"],
"dev_hepatocytes" : ["SERPINA1", "PRSS2", "SPINK1", "NRG3", "CASC19", "SERPINA3", "PRSS1"]
}

nucl_gene_set_dict = {
"nucl_50" : nucl_genes_50[:50],
"nucl_40" : nucl_genes_50[:40],
"nucl_30" : nucl_genes_50[:30],
"nucl_20" : nucl_genes_50[:20],
"nucl_10" : nucl_genes_50[:10],
"nucl_2" : nucl_genes_50[:2],
"nucl_1" : nucl_genes_50[:1]
}

In [3]:
def annotate_celltypes(df, leiden_key, key_added):
    df[key_added] = ["NA" for x in range(len(df.index))]
    for cluster in set(df[leiden_key]):
        subset_adata = df[df[leiden_key] == cluster]
        celltype = subset_adata[['score_VEC', 
                                 'score_PER', 
                                 'score_SMC', 
                                 'score_AD', 
                                 'score_SC', 
                                 'score_N', 
                                 'score_EEC', 
                                 'score_FB', 
                                 'score_L', 
                                 'score_MESO', 
                                 'score_MP', 
                                 'score_CM_nucl']].mean().idxmax().split("_")[1]
        df.loc[df[leiden_key] == cluster, key_added] = celltype
    return df[key_added].to_list()

In [4]:
def preproc(path=None):
    if path is None:
        print('provide path')
        return None
    
    adata = sc.read_mtx(f'{path}count_matrix.mtx')
    gene_data = pd.read_csv(f'{path}all_genes.csv')
    cell_meta = pd.read_csv(f'{path}cell_metadata.csv')
    # find genes with nan values and filter
    gene_data = gene_data[gene_data.gene_name.notnull()]
    notNa = gene_data.index
    notNa = notNa.to_list()
    adata = adata[:,notNa]
    adata.var = gene_data
    adata.var.set_index('gene_name', inplace=True)
    adata.var.index.name = None
    adata.var_names_make_unique()
    adata.obs = cell_meta
    adata.obs.set_index('bc_wells', inplace=True)
    adata.obs.index.name = None
    adata.obs_names_make_unique()
    
    for entry in celltype_gene_set_dict:
        adata.var[entry] = [True if x in celltype_gene_set_dict[entry] else False for x in adata.var.index]
        sc.pp.calculate_qc_metrics(adata, qc_vars=[entry], percent_top=None, log1p=False, inplace=True)
        sc.tl.score_genes(adata, gene_list = celltype_gene_set_dict[entry], score_name = f"score_{entry}")
    adata.obs["pct_counts_nonCM"] = adata.obs[['pct_counts_VEC', 'pct_counts_PER',  'pct_counts_SMC',  'pct_counts_AD',  'pct_counts_SC',  'pct_counts_N',  'pct_counts_EEC',  'pct_counts_FB',  'pct_counts_L',  'pct_counts_MESO',  'pct_counts_MP']].max(1)
    adata.obs["score_nonCM"] = adata.obs[['score_VEC', 'score_PER',  'score_SMC',  'score_AD',  'score_SC',  'score_N',  'score_EEC',  'score_FB',  'score_L',  'score_MESO',  'score_MP']].max(1)
    
    adata = adata[adata.obs.n_genes_by_counts >= 500]
    adata = adata[adata.obs.n_genes_by_counts <= 6000]
    adata = adata[adata.obs.pct_counts_MT <= 40]
    
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    
    for entry in nucl_gene_set_dict:
        adata.var[entry] = [True if x in nucl_gene_set_dict[entry] else False for x in adata.var.index]
        sc.pp.calculate_qc_metrics(adata, qc_vars=[entry], percent_top=None, log1p=False, inplace=True)
        sc.tl.score_genes(adata, gene_list = nucl_gene_set_dict[entry], score_name = f"score_{entry}")
    
    adata.raw = adata
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    sc.pp.filter_genes(adata, min_cells=10)
    adata = adata[:, adata.var.highly_variable]
    sc.pp.regress_out(adata, ['total_counts', 'pct_counts_MT'])
    sc.pp.scale(adata, max_value=10)
    
    sc.tl.pca(adata, svd_solver='arpack')
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
    sc.tl.umap(adata)
    
    sc.tl.leiden(adata, key_added="leiden_1_sample_before_filtering")
    sc.tl.leiden(adata, key_added="leiden_3_sample", resolution=3)        
    adata.obs["celltype_sample"] = annotate_celltypes(adata.obs.copy(), "leiden_3_sample", "celltype_sample")  
    
    return adata

In [ ]:
path = 'linnalab_data/minikit_test/spipe/newvolume/analysis/combined_results/EHT-337/DGE_filtered/'

adata_337 = preproc(path)
adata_337.write('337.h5ad')

path = 'linnalab_data/minikit_test/spipe/newvolume/analysis/combined_results/EHT-349/DGE_filtered/'

adata_349 = preproc(path)
adata_349.write('349.h5ad')

/home/users/johanneo/miniconda3/envs/python_env/lib/python3.12/site-packages/anndata/utils.py:260: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD115-1', 'SNORD115-2', 'SNORD115-3', 'SNORD115-4', 'SNORD115-5']
  warnings.warn(


computing score 'score_MT'
    finished: added
    'score_MT', score of gene set (adata.obs).
    350 total control genes are used. (0:00:00)
computing score 'score_MT_nucl'
    finished: added
    'score_MT_nucl', score of gene set (adata.obs).
    300 total control genes are used. (0:00:00)
computing score 'score_CM_cyto'
    finished: added
    'score_CM_cyto', score of gene set (adata.obs).
    50 total control genes are used. (0:00:00)
computing score 'score_CM_nucl'
    finished: added
    'score_CM_nucl', score of gene set (adata.obs).
    250 total control genes are used. (0:00:00)
computing score 'score_VEC'
    finished: added
    'score_VEC', score of gene set (adata.obs).
    349 total control genes are used. (0:00:00)
computing score 'score_PER'
    finished: added
    'score_PER', score of gene set (adata.obs).
    200 total control genes are used. (0:00:00)
computing score 'score_SMC'
    finished: added
    'score_SMC', score of gene set (adata.obs).
    300 total contr

/home/users/johanneo/miniconda3/envs/python_env/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
computing score 'score_nucl_50'
    finished: added
    'score_nucl_50', score of gene set (adata.obs).
    149 total control genes are used. (0:00:00)
computing score 'score_nucl_40'
    finished: added
    'score_nucl_40', score of gene set (adata.obs).
    150 total control genes are used. (0:00:00)
computing score 'score_nucl_30'
    finished: added
    'score_nucl_30', score of gene set (adata.obs).
    150 total control genes are used. (0:00:00)
computing score 'score_nucl_20'
    finished: added
    'score_nucl_20', score of gene set (adata.obs).
    150 total control genes are used. (0:00:00)
computing score 'score_nucl_10'
    finished: added
    'score_nucl_10', score of gene set (adata.obs).
    50 total control genes are used. (0:00:00)
computing score 'score_nucl_2'
    finished: added
    'score_nucl_2', score of gene set (adata.obs).
    50 total control genes are used. (0:00:00)
computing score 'score_nucl_1'
    finish

In [ ]:
sc.pl.umap(adata_337, 
           color=['leiden_3_sample', 
                  'celltype_sample', 
                  "score_dev_enteroendocrine", 
                  "score_dev_2", 
                  "score_dev_enterocytes", 
                  "score_dev_hepatocytes"],
           save='adata_337.png'
          )

In [ ]:
sc.pl.umap(adata_349, 
           color=['leiden_3_sample', 
                  'celltype_sample', 
                  "score_dev_enteroendocrine", 
                  "score_dev_2", 
                  "score_dev_enterocytes", 
                  "score_dev_hepatocytes"],
           save='adata_349.png'
          )